In [3]:
import dgl
import torch
from ogb.nodeproppred import DglNodePropPredDataset

In [20]:
og, _ = DglNodePropPredDataset(name='ogbn-products', root='/data/gangda/dgl')[0]

og.edata['w'] = torch.rand((og.num_edges()))
g = dgl.add_reverse_edges(og, copy_edata=True)
g.update_all(dgl.function.copy_e('w', 'weighted_degree'), dgl.function.sum('weighted_degree', 'weighted_degree'))
g

Graph(num_nodes=2449029, num_edges=247436304,
      ndata_schemes={'feat': Scheme(shape=(100,), dtype=torch.float32), 'weighted_degree': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'w': Scheme(shape=(), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [62]:
og.num_edges()

123718280

In [35]:
g.ndata['weighted_degree']

tensor([178.9047,  86.3341,  41.2517,  ...,  46.5904,   7.1525,  38.6172])

In [50]:
import dgl.sparse as dglsp

edge_index = torch.stack(g.edges())
N = g.num_nodes()
A = dglsp.spmatrix(indices=edge_index, val=g.edata['w'], shape=(N, N))
A

SparseMatrix(indices=tensor([[      0,  152857,       0,  ...,  162836, 2449028,   53324],
                             [ 152857,       0,   32104,  ..., 2449028,   53324, 2449028]]),
             values=tensor([0.4355, 0.1665, 0.8116,  ..., 0.4880, 0.6745, 0.8705]),
             shape=(2449029, 2449029), nnz=247436304)

In [52]:
D = dglsp.diag(A.sum(dim=1))
(D ** -1) @ A

SparseMatrix(indices=tensor([[      0,  152857,       0,  ...,  162836, 2449028,   53324],
                             [ 152857,       0,   32104,  ..., 2449028,   53324, 2449028]]),
             values=tensor([0.0024, 0.0001, 0.0045,  ..., 0.0013, 0.0175, 0.0008]),
             shape=(2449029, 2449029), nnz=247436304)

In [63]:
A.sum(dim=1)

tensor([178.9048,  86.3341,  41.2517,  ...,  46.5904,   7.1525,  38.6172])

In [60]:
d = g.ndata['weighted_degree'] - A.sum(dim=1)
d[d > 1e-6].shape

torch.Size([665742])

---

In [47]:
i = torch.tensor([[1, 1, 2],
                  [0, 2, 0]])
A = dglsp.spmatrix(i)  # 1.0 is default value for nnz elements.

print(A)
print("")
print("In dense format:")
print(A.to_dense())
A.csr()

SparseMatrix(indices=tensor([[1, 1, 2],
                             [0, 2, 0]]),
             values=tensor([1., 1., 1.]),
             shape=(3, 3), nnz=3)

In dense format:
tensor([[0., 0., 0.],
        [1., 0., 1.],
        [1., 0., 0.]])


(tensor([0, 0, 2, 3]), tensor([0, 2, 0]), tensor([0, 1, 2]))

In [67]:
t = torch.tensor([1, 0, 0, 0], dtype=torch.bool)
t & t

tensor([ True, False, False, False])

In [93]:
import re

input_str = 'd23-[10,13-16],e21-[01-03]'

input_str = input_str.split(',')
input_str

['d23-[10', '13-16]', 'e21-[01-03]']

In [144]:
import re

input_string = "a02-[01,06,20],a03-[01,06,15],b05-14,e02-[42,50,57-62,65-69,79],b10-13"
pattern = r'([a-z]\d{2})-\[(.*?)\]'
matches = re.findall(pattern, input_string)

hostnames = []
for prefix, values in matches:
    value_list = re.split(r'[,]', values)
    for value in value_list:
        if '-' in value:
            start, end = map(int, value.split('-'))
            for i in range(start, end + 1):
                hostnames.append(f'{prefix}-{str(i).zfill(2)}')
        else:
            hostnames.append(f'{prefix}-{value}')

# Handling single entries
single_entries = re.findall(r'([a-z]\d{2}-\d{2})', input_string)
hostnames.extend(single_entries)

print(hostnames)

['a02-01', 'a02-06', 'a02-20', 'a03-01', 'a03-06', 'a03-15', 'e02-42', 'e02-50', 'e02-57', 'e02-58', 'e02-59', 'e02-60', 'e02-61', 'e02-62', 'e02-65', 'e02-66', 'e02-67', 'e02-68', 'e02-69', 'e02-79', 'b05-14', 'b10-13']


In [146]:
hostnames.index('a02-20')

2

In [120]:
input_str = 'b04-[10-11],b09-[09-11]'

master_addr = input_str.split(',')[0]
match = re.match(r'(\w+-)\[(\w+)', master_addr)
if match:
    master_addr = match.group(1) + match.group(2)

master_addr

'b04-10'